# Простое управление потоком выполнения интрукций (flow control)  в R

Илья Кочергин 21.09.2016

В традиционных языках программирования команды управления потоком выполнения (циклы, условный оператор, оператор выбора) абсолютно  необходимы для постороения программ. В языке R без циклов можно обходиться  при помощи операций функционального программирования и векторных операций. Мы сначала  рассмотрим трациционные управляющие конструкции, а затем обсудим на что и в каких случаях их можно заменить.

## Составной оператор

Если вы пишете операторы на разных строках или на одной стороке через символ ";", то это будет последовательность отдельных операторов.

In [1]:
i<-1 ; i<-i+1 # не выводит значение
i
i+50

[1] 2

[1] 52

А последовательность операторов или выражений в фигурных скобках `{cmd1;cmd2;cmd3}` -- это составной оператор. Составной оператор можно применять  и управляющих конструкциях и для выработки значения

In [2]:
print({p <- 10;q <- 2; (p/q)^2 })

[1] 25


In [3]:
{p <- 10;q <- 2; (p/q)^2 }

[1] 25

Обратите внимание, что значение составного оператора -- это значение последнего выражения

## Видимое и невидимое выдаваемое значение

Выражение в R при выполении печатают свое значение, если у него атрибут видимости равен TRUE

In [4]:
2+2
withVisible(2+2)

[1] 4

$value
[1] 4

$visible
[1] TRUE

Оператор присваивания `p<-x` и функция `print(x)` помимо действия,   еще возвращают значение `x`, но они устанавливают аттрибут `visible=FALSE` чтобы на консоль еще раз не выводилось это значание.

In [5]:
p <- q <- 5
p

[1] 5

то есть оператор `q <- 5` вернул значеине `5` и оно присвоилось `p` но не выдалось на экран

In [6]:
r <- 5

In [7]:
(r <- 4+r)

[1] 9

Взяв выражение в круглые скобки, вы устаналниваете атрибут видимости в TRUE. Еще демонстрация с функцией `print()`:



In [8]:
print(1+print(2+print(3)))

[1] 3
[1] 5
[1] 6


Если вы хотите отменить автоматическую выдачу на экран значения выражения, то можно его окрузить функцей `invisible(выраж)` 

In [9]:
{p <- 10;q <- 2 ; invisible((p/q)^q )}

но значение вырабатывается:

In [10]:
cat("Результат:",{p <- 10;q <- 2 ; invisible((p/q)^q )})

Результат: 25

`cat(x1,x2, ...)` все свои аргументы преобразует в тип `character` и выдает один за другим.

## Условный оператор

`if (условие) {действия-если-TRUE} ` 

или 

`if (условие) {действия-если-TREE} else {действия-если-FALSE}` 

In [11]:
age <- 13
if(age < 20 && age>=10) { print("Teenager")}

[1] "Teenager"


In [12]:
age <- NA
if(age < 20 && age>=10) {
    print("Teenager")
}

ERROR: Error in if (age < 20 && age >= 10) {: пропущенное значение, а нужно TRUE/FALSE


In [16]:
age <- 23

if(age<11){
  print("Child")
} else if(age<20) {
  print("Teenager")    
} else if (age<45) {
  print("young")
} else {
  print("young forever")
}


[1] "young"


Можно использовать результат `if`

In [18]:
age <- 63

message <- if(age<11){
  "Child"
} else if(age<20) {
  "Teenager"    
} else if (age<45) {
  "young"
} else {
  "young forever"
}

print(message)

[1] "young forever"


## Оператор выбора switch (переключатель)

Оператор `switch(EXPR=выражение-selector , вариант1, вариант2, ... )` вычисляет выражение-selector и по его значнию выбирает и выполняет/вычиляет нужный вариант. 

### switch с числовым выражением выбора

In [14]:
ncolor=2
switch(EXPR = ncolor, "green","yellow","blue" )

[1] "yellow"

### switch со строковым выражением выбора

In [13]:
signal_name = "stop"
switch(EXPR = signal_name,go="green",ready="yellow",stop=,full_stop="red")

[1] "red"

## Циклы

Циклы позволяют повторять группу действий много раз. Есть 3 встроенных в язык R цикла:

* `repeat` (выход по команде `break`) 
* `while` (выход, когда условие продолжения станет ложным)
* `for` (переменной цикла по очереди присваивается элементы последовательности)


### repeat { block}

Следующий пример нужно выполнять **не в jupyter notebook** а в интерактивной среде (R, RStudio), поскольку 

`readline(prompt = "enter number (or q to exti): ")`  

выдает приглашение к вводу, запрашивает  и считывает ввод пользователя.

In [ ]:
repeat {
  answ <-  readline(prompt = "enter number (or q to exti): ")
  if(answ=="q" || answ =="Q") { print("goodbye!") ; break}
  num <- as.numeric(answ)
  print(sprintf("%g^3=%g",num,num^3))
}  
  

Возможный диалог с этой программой:

```
enter number (or q to exti): 11
[1] "11^3=1331"
enter number (or q to exti): 2
[1] "2^3=8"
enter number (or q to exti): 3
[1] "3^3=27"
enter number (or q to exti): a
[1] "NA^3=NA"
enter number (or q to exti): q
[1] "goodbye!"
Предупреждение:
в результате преобразования созданы NA 
```

Цикл `repeat` хорош для ситуаций, в которых сначала выполнятес действие (ввод пользователя, чтение строки из базы данных), а затем провераяется условие (считались ли данные или получен признак конца?).

## Цикл для каждого элемента последовательности ---  `for`

`for  (var in последовательность) {тело цикла}`


Переменная `var` по очереди принимает значение каждого элемента последовательноси  и выполнят с таким значением переменной группу операторов, которая составляет тело цикла.

In [19]:
for( i in list(a=1:10,b="abc",c=1:2))
{   
  print(i)
}

 [1]  1  2  3  4  5  6  7  8  9 10
[1] "abc"
[1] 1 2


Ещё пример:

In [21]:
l1  <- list(a=1:10,b="abc",c=1:2)

for( i in seq_along(l1))
{   
  cat(i," ",l1[[i]],"\n")
}

1   1 2 3 4 5 6 7 8 9 10 
2   abc 
3   1 2 


In [24]:
n=5
for (i in 1:n){
    cat("Гав! ")
}
cat('\n')

Гав! Гав! Гав! Гав! Гав! 


- - - 

*Замечание* Чем отличаетя `cat()` от `print()` ?

In [26]:
print("one\ntwo") 

[1] "one\ntwo"


In [27]:
cat("one\ntwo")

one
two

- - -
*Замечание* Чем отличаетя `cat()` от `paste()` ?

In [29]:
cat(1:5, letters[1:5])

1 2 3 4 5 a b c d e

In [30]:
paste(1:5, letters[1:5])

[1] "1 a" "2 b" "3 c" "4 d" "5 e"

In [31]:
paste0(1:5, letters[1:5])

[1] "1a" "2b" "3c" "4d" "5e"

Вложенные циклы  for -- распечатка матрицы:

In [32]:
x <- matrix(1:6, 2, 3)
for(i in seq_len(nrow(x))) {
  for(j in seq_len(ncol(x))) {
    cat(x[i, j]," ")
  }
  cat("\n")
}

1  3  5  
2  4  6  


In [33]:
print(x)

     [,1] [,2] [,3]
[1,]    1    3    5
[2,]    2    4    6


### Сумма вектора 3 способами

#### Способ 1: цикл for 

In [37]:
vv  <-  c(1,3,5,7)
vsum  <- 0 
for ( n in vv) { vsum  <- vsum+n }  
vsum

[1] 16

#### Способ 2: цикл while 

In [38]:
i <- 1 
vsum  <- 0 
while(i<=length(vv)) {
  vsum  <- vsum+vv[i] 
  i <- i+1
}
vsum

[1] 16

#### Способ 3: встроенная функция sum()

In [36]:
sum(vv)

[1] 16

### Пример из документации -- случайное блуждание

In [40]:
z <- 5
while(z >= 3 && z <= 10) {
  cat(z,' ')
  coin <- rbinom(1, 1, 0.5)
  if(coin == 1) { ## random walk
    z <- z + 1
  } else {
    z <- z - 1
  }
}


5  4  5  4  5  4  3  4  5  6  7  6  7  8  9  10  9  10  